# 1. 데이터 가져오기

In [ ]:
import os

data_folder = './data'

# all_files 변수에 data_folder경로에 있는 파일 리스트로 저장
all_files = os.listdir(data_folder)

# 파일 이름 저장할 리스트
wav_files = []

# all_files에 저장된 리스트 하나씩 가져오기
for f in all_files:
    # 조건문으로 뒤에 .WAV로 끝나는 것만 
    if f.endswith('.WAV'):
        # wav_files 리스트에 추가하기
        wav_files.append(f)
print(wav_files)

### 1-1 데이터 확인

In [ ]:
from IPython.display import Audio
import librosa
import matplotlib.pyplot as plt
from matplotlib import rc

# 한글 폰트 설정
rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False  # 음수 기호 깨짐 방지

# wav_files에 있는 파일 목록 하나씩 가져오기
for wav_file in wav_files:
    # os.path.join사용해서 data_folder와 wav_file을 조합하여 하나의 파일 경로를 생성
    file_path = os.path.join(data_folder, wav_file)

    # 음성 파일 로드
    y, sr = librosa.load(file_path, sr=None) # 원래 샘플링 레이트 유지
    # - y: 오디오 신호(시간 축에 따른 샘플 값)로, numpy배열 형식으로 반환
    # - sr: 샘플링 레이트(초당 샘플 수)
    
    # 파형 시각화
    plt.figure(figsize=(10,4))
    librosa.display.waveshow(y, sr=sr)
    plt.title("원본 데이터")
    plt.xlabel("시간")
    plt.ylabel("진폭") # 호의 "파형 높이"
    plt.show()

    # 음성 재생
    print("원본 파일 음성 재생")
    display(Audio(data=y, rate=sr))

# 2. 데이터 전처리

### 2-1 패커시브/하모닉 분리

In [ ]:
from IPython.display import Audio, display
import librosa
import librosa.display
import matplotlib.pyplot as plt
from matplotlib import rc
import os
import soundfile as sf

# 한글 폰트 설정
rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False  # 음수 기호 깨짐 방지

data_folder = './data'
output_folder = './cleaned_wav'

# 저장 폴더가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 데이터 폴더에서 .wav 파일 목록 가져오기
wav_files = [f for f in os.listdir(data_folder) if f.endswith('.WAV')]

# 파일 하나씩 처리
for wav_file in wav_files:
    # 파일 경로 생성
    file_path = os.path.join(data_folder, wav_file)

    # 음성 파일 로드
    y, sr = librosa.load(file_path, sr=None)  # 원래 샘플링 레이트 유지

    # 하모닉과 패커시브 분리
    D = librosa.stft(y)  # STFT 변환
    harmonic, percussive = librosa.decompose.hpss(D)  # H/P 분리
    y_harmonic = librosa.istft(harmonic, length=len(y))  # 원래 길이 유지
    y_percussive = librosa.istft(percussive, length=len(y))  # 원래 길이 유지

    # 하모닉 성분만 저장
    output_file_path = os.path.join(output_folder, f'cleaned_{wav_file}')
    sf.write(output_file_path, y_harmonic, sr)
    print(f"하모닉 성분 저장 완료: {output_file_path}")

    # 파형 시각화 (원본, 하모닉, 패커시브)
    plt.figure(figsize=(12, 8))

    # 원본 신호
    plt.subplot(3, 1, 1)
    librosa.display.waveshow(y, sr=sr, alpha=0.7)
    plt.title(f"원본 데이터 ({wav_file})")
    plt.xlabel("시간")
    plt.ylabel("진폭")

    # 하모닉 신호
    plt.subplot(3, 1, 2)
    librosa.display.waveshow(y_harmonic, sr=sr, alpha=0.7, color='orange')
    plt.title("하모닉 성분 (순수한 음)")
    plt.xlabel("시간")
    plt.ylabel("진폭")

    # 패커시브 신호
    plt.subplot(3, 1, 3)
    librosa.display.waveshow(y_percussive, sr=sr, alpha=0.7, color='green')
    plt.title("패커시브 성분 (잡음)")
    plt.xlabel("시간")
    plt.ylabel("진폭")

    plt.tight_layout()
    plt.show()

    # 음성 재생
    print(f"원본 파일 음성 재생 ({wav_file}):")
    display(Audio(data=y, rate=sr))

    print("하모닉 성분 음성 재생 (순수한 음):")
    display(Audio(data=y_harmonic, rate=sr))

    print("패커시브 성분 음성 재생 (잡음):")
    display(Audio(data=y_percussive, rate=sr))


- v0.1보다는 더 좋은 상태로 음성 분리 됨 
- 문제 : 패커시브로 분리된 음성 파일 깨끗하게 클릭음만 가져오지 못함(소 음성 소리 포함)

### 2-2 음성 패턴 분석

- 1. 음성 신호를 -> text로 변환
- 2. 1초에 클릭음이 몇번 발생하는지 출력해보기
- 3. 확인 됐다면 패턴을 분석해보기 (예. 7.24초만에 22번)
- 4. 평균 클릭음 패턴 분석을 했다면 그 초대로 끊고 범위를 -0.02, +0.02이런식으로 늘려서 클릭음 잡기
- 5. 음성 파일에 클릭음만 빼기 

2-3-1. 음성 신호를 -> text로 변환

In [ ]:
# 필요 라이브러리
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf

In [ ]:
# 데이터 폴더 경로
data_folder = './data'

# 폴더 내의 모든 파일 가져오기
audio_files = [f for f in os.listdir(data_folder) if f.endswith('.WAV')]

for file in audio_files:
    file_path = os.path.join(data_folder, file)

    # 오디오 파일 로드 (샘플링 주파수는 기본 22050Hz)
    y, sr = librosa.load(file_path, sr=None)
    
    # 주파수 분석 (Short-Time Fourier Transform)
    D = librosa.stft(y)
    D_db = librosa.amplitude_to_db(abs(D), ref=np.max)

    # 스펙트로그램 시각화
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(D_db, sr=sr, x_axis='time', y_axis='log')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f'Spectrogram of {file}')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.show()


스펙토그램 : 시간에 따른 주파수 성분의 강도 변화를 시각적으로 나타낸 그래프로, 오디오 신호의 주파수 패턴을 분석하는 데 사용됨.

색상: 신호 강도를 나타내며, 밝은 색(노랑)은 강한 신호, 어두운 색(보라/검정)은 약한 신호를 의미

스펙토그램 분석 
1. 노랑색 부분이 소 울음 소리 일 가능성 높음
2. 현재 0~150Hz에서 반복적인 패턴이 보임
3. 평균적으로 Hz는 최대가 16.384Hz정도임

In [ ]:
# 30초 기준 설정
target_duration = 30  # 초 단위
sampling_rate = 22050  # 샘플링 주파수 설정
hop_length = 512  # 프레임 간격 설정

# MFCC 추출 함수
def extract_mfcc(file_path):
    y, sr = librosa.load(file_path, sr=sampling_rate)

    # 원하는 30초 길이까지만 자르기
    max_samples = target_duration * sr  # 30초 샘플 크기 계산
    if len(y) > max_samples:
        y = y[:max_samples]  # 30초까지만 사용

    # MFCC 추출
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=hop_length)
    return mfccs, sr

# 오디오 파일 처리 및 시각화
data_folder = './data'  # 오디오 파일 폴더 경로
audio_files = [f for f in os.listdir(data_folder) if f.endswith('.WAV')]

for file in audio_files:
    file_path = os.path.join(data_folder, file)
    mfcc_features, sr = extract_mfcc(file_path)

    # 시각화
    plt.figure(figsize=(12, 6))
    librosa.display.specshow(mfcc_features, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
    plt.colorbar(label='MFCC Coefficients')
    plt.title(f'MFCC of {file}')
    plt.xlabel('Time (s)')
    plt.ylabel("Frequency (Mel)")
    plt.grid()
    plt.show()

- MFCC : 인간의 청각 특성을 모방하여 오디오 신호의 주파수 정보를 멜 스케일로 변환한 후 특징을 추출하는 방법으로, 음성 및 오디오 인식에 널리 사용됨.
- 멜 스케일 : 멜 스케일은 주파수를 인간의 인식에 맞게 비선형적으로 변환한 척도
- 붉은색(Red): 높은 MFCC 값을 나타내며, 특정 주파수 대역에서 강한 에너지를 의미.
- 푸른색(Blue): 낮은 MFCC 값을 나타내며, 에너지가 약한 영역.

MFCC 분석
- 0~512 Mel (저주파)
- > 반복적인 패턴이 관찰되며, 주기적인 저주파 소음(기계 소음, 배경 잡음) 존재 가능성.
- 512~1024 Mel (저주파중주파)
- > 낮은 목소리의 지속적인 성분이 포함될 수 있으며, 배경 소음과 혼합 가능성.
- 1024~2048 Mel (중주파)
- > 음성의 주요 포먼트가 나타나는 구간이며, 특정 패턴이 유지됨.
- 2048~4096 Mel (중고주파)
- > 음성의 자음 특성 혹은 환경 잡음의 성분이 있을 가능성.
- 4096~8192 Mel (고주파)
- > 고주파 노이즈 성분이 일정하게 포함되며, 음향 신호보다는 잡음일 가능성이 있음.

In [ ]:
output_folder = './mfcc_features'  # 특징 저장 폴더
os.makedirs(output_folder, exist_ok=True)

for file in audio_files:
    file_path = os.path.join(data_folder, file)
    mfcc_features, sr = extract_mfcc(file_path)

    # MFCC 데이터를 CSV로 저장
    mfcc_df = pd.DataFrame(mfcc_features.T)  # 전치하여 시간축 기준으로 정렬
    output_csv = os.path.join(output_folder, f"{file}.csv")
    mfcc_df.to_csv(output_csv, index=False)
    print(f"MFCC 저장 완료: {output_csv}")

- 이 방법으론 할 수 없다고 피드백 받음 (중단)
> 안되는 이유 : 클릭음뿐만 아니라 해당 구간의 원본 음성까지 함께 제거될 수 있음